##### Andrés Felipe Rengifo. 

In [156]:
import pandas as pd
import os 
import numpy as np
from datetime import date

In [157]:
user= os.getlogin( )
root= "C:\\Users\\" +user+ "\\Universidad de los Andes\\UK PACT- Giro Zero - GiroZero GPS - GiroZero GPS"
empresas_folders= [f for f in os.listdir(root)] 
empresas_folders.pop(0)
empresas_folders.pop(0)
empresas_folders.pop(0)
empresas_folders

['BOTERO SOTO (NOEL)(NUTRESA)',
 'C&S LOGISTICA',
 'CEMEX',
 'CENTROLIMA',
 'COLTANQUES',
 'COTRANSCOL',
 'DITRANSA',
 'HDL - Bimbo',
 'HDL - Colsubsidio',
 'PLEXA',
 'POSTOBON',
 'RENTING COLOMBIA',
 'SOLISTICA',
 'TDH',
 'TENARIS',
 'TRANSPORTES SANCHEZ POLO (TSP)',
 'Z. RESUMEN PLACAS']

In [158]:
empresas_folders

['BOTERO SOTO (NOEL)(NUTRESA)',
 'C&S LOGISTICA',
 'CEMEX',
 'CENTROLIMA',
 'COLTANQUES',
 'COTRANSCOL',
 'DITRANSA',
 'HDL - Bimbo',
 'HDL - Colsubsidio',
 'PLEXA',
 'POSTOBON',
 'RENTING COLOMBIA',
 'SOLISTICA',
 'TDH',
 'TENARIS',
 'TRANSPORTES SANCHEZ POLO (TSP)',
 'Z. RESUMEN PLACAS']

#####  Botero Soto

In [159]:
empresa_dir= os.path.join(root, empresas_folders[0])
data_folders= [f for f in os.listdir(empresa_dir) if "BRUTOS" in f] 
folder_brutos= os.path.join(empresa_dir, data_folders[0])
folder_brutos

'C:\\Users\\a.rengifoj\\Universidad de los Andes\\UK PACT- Giro Zero - GiroZero GPS - GiroZero GPS\\BOTERO SOTO (NOEL)(NUTRESA)\\1. DATOS BRUTOS'

In [160]:
### dada la estructura de los datos, permite obtener la cantidad de archivos 

data_folders=  [f for f in os.listdir(folder_brutos)]

files= []
for i in range(len(data_folders)):
    tem_folder= os.path.join(folder_brutos, data_folders[i])
    tem_files= [f for f in os.listdir(tem_folder)]
    for j in range(len(tem_files)):
        tem_path=os.path.join(tem_folder, tem_files[j])
        files.append(tem_path)
tem_path

'C:\\Users\\a.rengifoj\\Universidad de los Andes\\UK PACT- Giro Zero - GiroZero GPS - GiroZero GPS\\BOTERO SOTO (NOEL)(NUTRESA)\\1. DATOS BRUTOS\\1. DATOS BRUTOS HASTA 11-05-2023\\Información GPS Noel 2022 (Botero Soto).xlsx'

In [161]:
gps= pd.read_excel(tem_path)

#### Velocidades

In [162]:
### velocidad
velocity= gps.groupby("Vehículo FO")[['VELOCIDAD']].agg(['mean', 'min', 'max'])
velocity.reset_index(inplace=True)
colnames= [ f[0] +"_" + f[1]  for f in  velocity.columns]
velocity.columns= colnames
velocity.rename(columns={"Vehículo FO_": "PLACA"}, inplace= True)

velocity['Rango Velocidad']= ""
for x in range(len(velocity)):
     a=velocity.loc[x, 'VELOCIDAD_min']
     b=velocity.loc[x, 'VELOCIDAD_max']
     velocity.loc[x, 'Rango Velocidad']= "("+ str(a) + "-" + str(b) + ")"
            
            
velocity

,PLACA,VELOCIDAD_mean,VELOCIDAD_min,VELOCIDAD_max,Rango Velocidad
0,ESR348,33.537335,0.0,91.0,(0.0-91.0)
1,GDX600,12.314992,0.0,70.0,(0.0-70.0)
2,GDY060,27.709486,0.0,74.0,(0.0-74.0)
3,GDY915,9.248756,0.0,73.0,(0.0-73.0)
4,GUR562,30.474227,0.0,78.0,(0.0-78.0)
...,...,...,...,...,...
86,WLW708,32.511991,0.0,82.0,(0.0-82.0)
87,WPT537,30.248020,0.0,89.0,(0.0-89.0)
88,XKH903,8.848432,0.0,68.0,(0.0-68.0)
89,XVH092,25.652353,0.0,60.0,(0.0-60.0)


In [173]:
velocity= velocity[['PLACA', 'VELOCIDAD_mean', "Rango Velocidad" ]]

In [174]:
velocity

,PLACA,VELOCIDAD_mean,Rango Velocidad
0,ESR348,33.537335,(0.0-91.0)
1,GDX600,12.314992,(0.0-70.0)
2,GDY060,27.709486,(0.0-74.0)
3,GDY915,9.248756,(0.0-73.0)
4,GUR562,30.474227,(0.0-78.0)
...,...,...,...
86,WLW708,32.511991,(0.0-82.0)
87,WPT537,30.248020,(0.0-89.0)
88,XKH903,8.848432,(0.0-68.0)
89,XVH092,25.652353,(0.0-60.0)


##### DATOS VALIDOS

In [164]:
gps['velocidad_valida']= 1
gps.loc[gps['VELOCIDAD']> 125, 'velocidad_valida']= 0


#lat_lon= pd.DataFrame()
for var in ['LONGITUD_X', 'LATITUD_Y']:
        varname= var + "_valida"
        gps[varname]= 1
        if var== "LONGITUD_X":
                gps.loc[abs(gps[var])> 180, 'velocidad_valida']= 0
        elif var== "LATITUD_Y":
                gps.loc[abs(gps[var])> 90, 'velocidad_valida']= 0

gps['obs_valida']= 1
gps.loc[(gps['velocidad_valida']== 0) | (gps['LONGITUD_X_valida']== 0) | (gps['LATITUD_Y_valida']== 0), 'obs_valida']= 0

obs_validas= gps.groupby("Vehículo FO")[['obs_valida']].agg(['mean', "sum"])
obs_validas.reset_index(inplace=True)
obs_validas.columns= ["PLACA", "obs_validas (%)", "obs_validas total"]

#### Numero de registros por placa.

In [165]:
### numero de registros por camión
regis=gps.groupby("Vehículo FO")[['Vehículo FO']].agg('count')
regis.columns= ["CANT.REGISTROS"]
regis.reset_index(inplace=True)
regis.head(15)

,Vehículo FO,CANT.REGISTROS
0,ESR348,5665
1,GDX600,1254
2,GDY060,506
3,GDY915,201
4,GUR562,1552
5,JYM629,3
6,JYO016,1925
7,KSO640,1103
8,LJW805,2902
9,LPR060,1599


#### latitud y Longitus

In [166]:
### latitud longitud
t=0
#lat_lon= pd.DataFrame()
for var in ['LONGITUD_X', 'LATITUD_Y']:
      tem_data=  gps.groupby("Vehículo FO")[[var]].agg(['min', 'max'])
      tem_data.columns= tem_data.columns.droplevel()
      tem_data.reset_index(inplace=True)
      col_min= var + '_min'
      col_max= var + '_max'
      tem_data.columns= ['Vehículo FO',col_min , col_max]
      if t==0:
            lat_lon=tem_data.copy()
      else: 
            lat_lon= pd.merge(lat_lon, tem_data, on= 'Vehículo FO') 
      t= t+1
lat_lon['Rango Longitud']= ""
lat_lon['Rango Latitud']= ""
for x in range(len(lat_lon)):
      for var in ['LONGITUD_X', 'LATITUD_Y']:
            col_min= var + '_min'
            col_max= var + '_max'
            a=lat_lon.loc[x, col_min]
            b=lat_lon.loc[x, col_max]
            if var== 'LONGITUD_X':
                  lat_lon.loc[x, 'Rango Longitud']= "("+ str(a) + "," + str(b) + ")"
            elif var== 'LATITUD_Y':
                  lat_lon.loc[x, 'Rango Latitud']= "("+ str(a) + "," + str(b) + ")"


In [168]:
revision_data= pd.merge(lat_lon[["Vehículo FO", "Rango Longitud", "Rango Latitud" ]], regis, on= "Vehículo FO" )

In [169]:
### Fecha
today = date.today()
revision_data["Fecha Revisión"]= today
### id
a= len(revision_data)+ 1
revision_data["ID Archivo-Placa"]= range(1, a)
### nombre archivo-pestaña
revision_data["Nombre Archivo-Pestaña"]= "Información GPS Noel 2022 (Botero Soto).xlsx" + "HOJA 1"
# PLACA
revision_data = revision_data.rename(columns={'Vehículo FO': 'PLACA'})
#### ESPECIFICAR EL PROBLEMA
#revision_data["ESPECIFICAR PROBLEMA"]= "" 
#revision_data.loc[revision_data['¿CUMPLE FORMATO?']== 0, 'ESPECIFICAR PROBLEMA'] = "Latitud que pasan de -90, 90 - Longitudes que pasan -180, 180"


In [198]:
#### Agregados
agregado= pd.DataFrame( columns= [ 'TOTAL ARCHIVOS/BD',  'ARCHIVOS/BD CORRECTOS', 'TOTAL VEHÍCULOS' , 'TOTAL VEHÍCULOS CORRECTOS',	 'TOTAL REGISTROS',	'TOTAL REGISTROS CORRECTOS', 'PORCENTAGE REGISTROS CORRECTOS' ])

### total archivos
agregado.loc[1, 'TOTAL ARCHIVOS/BD' ]= 1

# total vehículos
a= len(revision_data)
agregado.loc[1, 'TOTAL VEHÍCULOS' ]= a

# archivos correctos
agregado.loc[1, 'ARCHIVOS/BD CORRECTOS' ]= 1

# Total vehículos correctos
agregado.loc[1, 'TOTAL VEHÍCULOS CORRECTOS' ]= a

## total registros
a= revision_data['CANT.REGISTROS'].sum()
agregado.loc[1, 'TOTAL REGISTROS' ]= a

### total registrs válidos

a= revision_data['obs_validas total'].sum()
agregado.loc[1, 'TOTAL REGISTROS CORRECTOS' ]= a

agregado.loc[1, 'PORCENTAGE REGISTROS CORRECTOS' ]= agregado.loc[1, 'TOTAL REGISTROS CORRECTOS' ]/agregado.loc[1, 'TOTAL REGISTROS' ]


In [199]:
agregado

,TOTAL ARCHIVOS/BD,ARCHIVOS/BD CORRECTOS,TOTAL VEHÍCULOS,TOTAL VEHÍCULOS CORRECTOS,TOTAL REGISTROS,TOTAL REGISTROS CORRECTOS,PORCENTAGE REGISTROS CORRECTOS
1,1,1,91,91,194586,194469,0.999399


In [200]:

out_folders_1= [f for f in os.listdir(empresa_dir) if "REVISADOS" in f] 
temp_path= os.path.join(empresa_dir, out_folders_1[0])
out_folders_2=  [f for f in os.listdir(temp_path) if "PLACAS" in f]  
temp_path= os.path.join(temp_path,out_folders_2[0] )
file_name= empresas_folders[0] + " TRAZABILIDAD_code.xlsx"
out_file=  os.path.join(temp_path, file_name) 

writer = pd.ExcelWriter(out_file, engine="xlsxwriter")
revision_data.to_excel(writer, index=False, sheet_name= "desagregado")
agregado.to_excel(writer, index=False, sheet_name= "agregado")

writer.close()
out_file

'C:\\Users\\a.rengifoj\\Universidad de los Andes\\UK PACT- Giro Zero - GiroZero GPS - GiroZero GPS\\BOTERO SOTO (NOEL)(NUTRESA)\\2. DATOS REVISADOS\\1. INFORMACIÓN PLACAS\\BOTERO SOTO (NOEL)(NUTRESA) TRAZABILIDAD_code.xlsx'